## Load the Data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [3]:
file_dir = r"C:\Users\slab\Desktop\Stage1\data\Device_AB00035.csv"

df = pd.read_csv(file_dir, low_memory=False)

## Data Preprocessing

In [4]:
# Date preprocessing
# 只留下目標船資料
mmsi_target = 416426000


df_filtered = df[df['MMSI'] == mmsi_target].copy()
# 1. 去掉不合理經緯度
df_filtered = df_filtered[
    (df_filtered['Lat'] >= -90) & (df_filtered['Lat'] <= 90) &
    (df_filtered['Long'] >= -180) & (df_filtered['Long'] <= 180)
].copy()
# 2. 把lat, long 都換成float, Receivetime 換成datetime64
df_filtered['Lat'] = df_filtered['Lat'].astype(float)
df_filtered['Long'] = df_filtered['Long'].astype(float)
df_filtered['Timestamp'] = pd.to_datetime(df_filtered['Timestamp'], errors='coerce')
# 3. 丟掉缺失經緯度的欄位
df_filtered = df_filtered.dropna(subset=['Lat', 'Long', 'Timestamp'])
df_filtered['Long_360'] = df_filtered['Long'] % 360
# Rot 欄位轉成數字
df_filtered['Rot'] = pd.to_numeric(df_filtered['Rot'], errors='coerce')

In [5]:
df_filtered.dtypes

PKY                          int64
Timestamp           datetime64[ns]
MsgId                      float64
OriginMessage               object
MMSI                       float64
Lat                        float64
Long                       float64
Cog                        float64
Rot                        float64
Hdg                        float64
Sog                        float64
Error                       object
NavigationStatus           float64
ReceiveTime                 object
IsSend                       int64
SendTime                   float64
Long_360                   float64
dtype: object

In [6]:
df_filtered['Timestamp'].head(20)

0    1970-01-01 05:37:20.421000003
1    1970-01-01 05:37:20.421000013
2    1970-01-01 05:37:20.421000024
3    1970-01-01 05:37:20.421000033
4    1970-01-01 05:37:20.421000052
5    1970-01-01 05:37:20.421000103
6    1970-01-01 05:37:20.421000113
7    1970-01-01 05:37:20.421000124
8    1970-01-01 05:37:20.421000133
9    1970-01-01 05:37:20.421000143
10   1970-01-01 05:37:20.421000152
11   1970-01-01 05:37:20.421000203
12   1970-01-01 05:37:20.421000213
13   1970-01-01 05:37:20.421000224
14   1970-01-01 05:37:20.421000233
15   1970-01-01 05:37:20.421000243
16   1970-01-01 05:37:20.421000252
17   1970-01-01 05:37:20.421000303
18   1970-01-01 05:37:20.421000313
19   1970-01-01 05:37:20.421000324
Name: Timestamp, dtype: datetime64[ns]

In [ ]:
# 看一下資料資間落在甚麼區段
df_filtered['ReceiveTime'].hist()
 # pandas 內建 hist
plt.show()

## Set the Threshold

In [ ]:
# 確保時間格式正確 & 排序
df_filtered['ReceiveTime'] = pd.to_datetime(df_filtered['ReceiveTime'])
df_filtered = df_filtered.sort_values('ReceiveTime').reset_index(drop=True)



In [ ]:
# 停泊判斷條件
# Lat, Long不要變化太大
# Sog < 0.5 knots
# sort the time
# filter temporary mooring : < 30min
# 計算經緯度差分
df_filtered['Delta_Lat'] = df_filtered['Lat'].diff().abs()
df_filtered['Delta_Long'] = df_filtered['Long_360'].diff().abs()

# 計算時間差（秒）
# 拿來給delta_lat/long除，也許資料時間間隔不是幾秒內，這樣下面判斷會失效
df_filtered['Delta_Time'] = df_filtered['ReceiveTime'].diff().dt.total_seconds()

#設置一個短時間的閥值
short_time_thresh = 1.0

# 用 np.where 來處理加權 delta
df_filtered["Adj_Delta_Lat"] = np.where(
    df_filtered["Delta_Time"] > short_time_thresh,
    df_filtered["Delta_Lat"] / df_filtered["Delta_Time"],  # 時間長才除以秒數
    df_filtered["Delta_Lat"]  # 時間短就直接用距離
)

df_filtered["Adj_Delta_Long"] = np.where(
    df_filtered["Delta_Time"] > short_time_thresh,
    df_filtered["Delta_Long"] / df_filtered["Delta_Time"],
    df_filtered["Delta_Long"]
)

# 設定閾值
sog_threshold = 0.5
rate_threshold = 0.0001  # 約 100m，可依需求調整

# 修正後的 Delta 判斷停泊
# 接著使用這個修正後的 delta 判斷停泊
df_filtered["Is_Stop"] = (
    (df_filtered["Sog"] < sog_threshold) &
    #(df_filtered["Rot"].abs() < rot_threshold) &
    (df_filtered["Adj_Delta_Lat"].abs() < rate_threshold) &
    (df_filtered["Adj_Delta_Long"].abs() < rate_threshold)
)

In [ ]:
df_filtered['Is_Stop']

In [ ]:
# 1. 計算 True / False 的數量
print(df_filtered['Is_Stop'].value_counts())

# 2. 顯示比例
print(df_filtered['Is_Stop'].value_counts(normalize=True))

# 3. 畫成柱狀圖
import matplotlib.pyplot as plt

df_filtered['Is_Stop'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.xticks([0,1], ['False (航行)', 'True (停泊)'], rotation=0)
plt.ylabel('Count')
plt.title('停泊狀態 True/False 分布')
plt.show()

# 4. 畫成折線圖看連續停泊的分布（可視化時間序列）
plt.figure(figsize=(15,3))
plt.plot(df_filtered['ReceiveTime'], df_filtered['Is_Stop'].astype(int), marker='o', linestyle='-', markersize=2)
plt.ylabel('Is_Stop (1=True, 0=False)')
plt.xlabel('ReceiveTime')
plt.title('停泊狀態時間序列')
plt.show()

In [ ]:
# 找停泊區段
stop_segments = []
current_start = None

for i in range(len(df_filtered)):
    if df_filtered.loc[i, 'Is_Stop']:
        if current_start is None:
            current_start = df_filtered.loc[i, 'ReceiveTime']
    else:
        if current_start is not None:
            # 結束一段停泊
            current_end = df_filtered.loc[i-1, 'ReceiveTime']
            duration = (current_end - current_start).total_seconds()
            stop_segments.append((current_start, current_end, duration))
            current_start = None

# 如果最後一段停泊持續到資料尾端
if current_start is not None:
    current_end = df_filtered['ReceiveTime'].iloc[-1]
    duration = (current_end - current_start).total_seconds()
    stop_segments.append((current_start, current_end, duration))

In [ ]:
# 轉成 DataFrame
stops_df = pd.DataFrame(stop_segments, columns=['StartTime', 'EndTime', 'Duration_sec'])

# 過濾掉 < 30 分鐘的停泊
stops_df = stops_df[stops_df['Duration_sec'] >= 1800].reset_index(drop=True)

print("停泊區段：")
print(stops_df)

# 計算總停泊時間
total_stop_time = stops_df['Duration_sec'].sum()

# 總觀測時間
total_time = (df_filtered['ReceiveTime'].max() - df_filtered['ReceiveTime'].min()).total_seconds()

# 航行時間 = 總時間 - 停泊時間
sailing_time = total_time - total_stop_time

print(f"總停泊時間 (hr): {total_stop_time/3600:.2f}")
print(f"航行時間 (hr): {sailing_time/3600:.2f}")
print(f"停泊比例: {total_stop_time/total_time:.2%}")
print(f"航行比例: {sailing_time/total_time:.2%}")

In [ ]:
## Load the Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

file_dir = r"C:\Users\slab\Desktop\Stage1\data\Device_AB00006.csv"
df = pd.read_csv(file_dir, low_memory=False)

## Data Preprocessing
mmsi_target = 416464000


df_filtered = df[df['MMSI'] == mmsi_target].copy()

# 去掉不合理經緯度
df_filtered = df_filtered[
    (df_filtered['Lat'] >= -90) & (df_filtered['Lat'] <= 90) &
    (df_filtered['Long'] >= -180) & (df_filtered['Long'] <= 180)
].copy()

# 型態轉換
df_filtered['Lat'] = df_filtered['Lat'].astype(float)
df_filtered['Long'] = df_filtered['Long'].astype(float)
df_filtered['ReceiveTime'] = pd.to_datetime(df_filtered['ReceiveTime'], errors='coerce')
df_filtered = df_filtered.dropna(subset=['Lat', 'Long', 'ReceiveTime'])
df_filtered['Long_360'] = df_filtered['Long'] % 360
df_filtered['Rot'] = pd.to_numeric(df_filtered['Rot'], errors='coerce')

# 檢查資料落點
df_filtered['ReceiveTime'].hist()
plt.show()

# 排序時間
df_filtered = df_filtered.sort_values('ReceiveTime').reset_index(drop=True)

## 停泊判斷條件
# 計算經緯度差分
df_filtered['Delta_Lat'] = df_filtered['Lat'].diff().abs()
df_filtered['Delta_Long'] = df_filtered['Long_360'].diff().abs()
# 計算時間差（秒）
df_filtered['Delta_Time'] = df_filtered['ReceiveTime'].diff().dt.total_seconds()

# 閾值設定
short_time_thresh = 1.0       # 短時間判斷用
sog_threshold = 0.5           # knots
rate_threshold = 0.0001       # 約 100m
max_gap_sec = 30*60           # 30 分鐘
large_delta_thresh = 0.01     # 約 1km，可調整

# 判斷「時間跨度長且位置變化大」
is_large_gap_and_move = (df_filtered['Delta_Time'] > max_gap_sec) & \
                        ((df_filtered['Delta_Lat'].abs() > large_delta_thresh) | 
                         (df_filtered['Delta_Long'].abs() > large_delta_thresh))

# 計算加權 delta
df_filtered['Adj_Delta_Lat'] = np.where(
    is_large_gap_and_move,
    df_filtered['Delta_Lat'],  # 時間跨度長且移動大 → 不除以時間
    np.where(
        df_filtered['Delta_Time'] > short_time_thresh,
        df_filtered['Delta_Lat'] / df_filtered['Delta_Time'],
        df_filtered['Delta_Lat']
    )
)

df_filtered['Adj_Delta_Long'] = np.where(
    is_large_gap_and_move,
    df_filtered['Delta_Long'],  # 同上
    np.where(
        df_filtered['Delta_Time'] > short_time_thresh,
        df_filtered['Delta_Long'] / df_filtered['Delta_Time'],
        df_filtered['Delta_Long']
    )
)

# 停泊判斷
df_filtered['Is_Stop'] = (
    (df_filtered['Sog'] < sog_threshold) &
    (df_filtered['Adj_Delta_Lat'].abs() < rate_threshold) &
    (df_filtered['Adj_Delta_Long'].abs() < rate_threshold)
)

# True/False 分布
print(df_filtered['Is_Stop'].value_counts())
print(df_filtered['Is_Stop'].value_counts(normalize=True))

# 畫柱狀圖
df_filtered['Is_Stop'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.xticks([0,1], ['False (航行)', 'True (停泊)'], rotation=0)
plt.ylabel('Count')
plt.title('停泊狀態 True/False 分布')
plt.show()

# 畫時間序列
plt.figure(figsize=(15,3))
plt.plot(df_filtered['ReceiveTime'], df_filtered['Is_Stop'].astype(int), marker='o', linestyle='-', markersize=2)
plt.ylabel('Is_Stop (1=True, 0=False)')
plt.xlabel('ReceiveTime')
plt.title('停泊狀態時間序列')
plt.show()

# 找停泊區段
stop_segments = []
current_start = None

for i in range(len(df_filtered)):
    if df_filtered.loc[i, 'Is_Stop']:
        if current_start is None:
            current_start = df_filtered.loc[i, 'ReceiveTime']
    else:
        if current_start is not None:
            current_end = df_filtered.loc[i-1, 'ReceiveTime']
            duration = (current_end - current_start).total_seconds()
            stop_segments.append((current_start, current_end, duration))
            current_start = None

# 如果最後一段停泊持續到資料尾端
if current_start is not None:
    current_end = df_filtered['ReceiveTime'].iloc[-1]
    duration = (current_end - current_start).total_seconds()
    stop_segments.append((current_start, current_end, duration))

# 轉成 DataFrame 並過濾短停泊
stops_df = pd.DataFrame(stop_segments, columns=['StartTime', 'EndTime', 'Duration_sec'])
stops_df = stops_df[stops_df['Duration_sec'] >= 1800].reset_index(drop=True)
print("停泊區段：")
print(stops_df)

# 計算總停泊時間與航行時間
total_stop_time = stops_df['Duration_sec'].sum()
total_time = (df_filtered['ReceiveTime'].max() - df_filtered['ReceiveTime'].min()).total_seconds()
sailing_time = total_time - total_stop_time

print(f"總停泊時間 (hr): {total_stop_time/3600:.2f}")
print(f"航行時間 (hr): {sailing_time/3600:.2f}")
print(f"停泊比例: {total_stop_time/total_time:.2%}")
print(f"航行比例: {sailing_time/total_time:.2%}")

# 畫停泊/航行時間比例圖
plt.figure(figsize=(6,6))
plt.pie([total_stop_time, sailing_time], labels=['停泊', '航行'], autopct='%1.1f%%', colors=['salmon','skyblue'], startangle=90)
plt.title('停泊/航行時間比例')
plt.show()


## 整合所有程式碼

In [ ]:
## Load the Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns

file_dir = r"C:\Users\slab\Desktop\Stage1\data\Device_AB00035.csv"
df = pd.read_csv(file_dir, low_memory=False)

## Data Preprocessing
# 只留下目標船資料
mmsi_target = 416426000

df_filtered = df[df['MMSI'] == mmsi_target].copy()

# 去掉不合理經緯度
df_filtered = df_filtered[
    (df_filtered['Lat'] >= -90) & (df_filtered['Lat'] <= 90) &
    (df_filtered['Long'] >= -180) & (df_filtered['Long'] <= 180)
].copy()

# 將 lat, long 轉 float，ReceiveTime 轉 datetime64
df_filtered['Lat'] = df_filtered['Lat'].astype(float)
df_filtered['Long'] = df_filtered['Long'].astype(float)
df_filtered['ReceiveTime'] = pd.to_datetime(df_filtered['ReceiveTime'], errors='coerce')

# 丟掉缺失資料
df_filtered = df_filtered.dropna(subset=['Lat', 'Long', 'ReceiveTime'])
df_filtered['Long_360'] = df_filtered['Long'] % 360
df_filtered['Rot'] = pd.to_numeric(df_filtered['Rot'], errors='coerce')

# 確保時間排序
df_filtered = df_filtered.sort_values('ReceiveTime').reset_index(drop=True)

# 計算經緯度差分
df_filtered['Delta_Lat'] = df_filtered['Lat'].diff().abs()
df_filtered['Delta_Long'] = df_filtered['Long_360'].diff().abs()

# 計算時間差（秒）
df_filtered['Delta_Time'] = df_filtered['ReceiveTime'].diff().dt.total_seconds()

# 設置短時間閥值（秒）與 delta 閥值
short_time_thresh = 1.0   # 秒
sog_threshold = 0.5       # knots
rate_threshold = 0.0001   # 經緯度差閾值，約100m，可調整

# 修正 delta 經緯度
df_filtered['Adj_Delta_Lat'] = np.where(
    df_filtered['Delta_Time'] > short_time_thresh,
    df_filtered['Delta_Lat'] / df_filtered['Delta_Time'],
    df_filtered['Delta_Lat']
)
df_filtered['Adj_Delta_Long'] = np.where(
    df_filtered['Delta_Time'] > short_time_thresh,
    df_filtered['Delta_Long'] / df_filtered['Delta_Time'],
    df_filtered['Delta_Long']
)

# 停泊判斷
df_filtered['Is_Stop'] = (
    (df_filtered['Sog'] < sog_threshold) &
    (df_filtered['Adj_Delta_Lat'].abs() < rate_threshold) &
    (df_filtered['Adj_Delta_Long'].abs() < rate_threshold)
)

# 檢查 True/False 分布
print(df_filtered['Is_Stop'].value_counts())
print(df_filtered['Is_Stop'].value_counts(normalize=True))

# 視覺化
df_filtered['Is_Stop'].value_counts().plot(kind='bar', color=['skyblue','salmon'])
plt.xticks([0,1], ['False (航行)','True (停泊)'], rotation=0)
plt.ylabel('Count')
plt.title('停泊狀態 True/False 分布')
plt.show()

plt.figure(figsize=(15,3))
plt.plot(df_filtered['ReceiveTime'], df_filtered['Is_Stop'].astype(int), marker='o', linestyle='-', markersize=2)
plt.ylabel('Is_Stop (1=True, 0=False)')
plt.xlabel('ReceiveTime')
plt.title('停泊狀態時間序列')
plt.show()

# -----------------------------
# 找停泊區段（改良版，容忍短暫中斷）
# -----------------------------
max_gap_sec = 120  # 允許 2 分鐘內 False 不切斷停泊

stop_segments = []
current_start = None
last_stop_time = None

for i in range(len(df_filtered)):
    if df_filtered.loc[i, 'Is_Stop']:
        if current_start is None:
            current_start = df_filtered.loc[i, 'ReceiveTime']
        last_stop_time = df_filtered.loc[i, 'ReceiveTime']
    else:
        if current_start is not None:
            gap = (df_filtered.loc[i, 'ReceiveTime'] - last_stop_time).total_seconds()
            if gap > max_gap_sec:
                # 結束停泊段
                current_end = last_stop_time
                duration = (current_end - current_start).total_seconds()
                stop_segments.append((current_start, current_end, duration))
                current_start = None
                last_stop_time = None

# 最後一段停泊若持續到資料尾端
if current_start is not None:
    current_end = df_filtered['ReceiveTime'].iloc[-1]
    duration = (current_end - current_start).total_seconds()
    stop_segments.append((current_start, current_end, duration))

# 轉成 DataFrame
stops_df = pd.DataFrame(stop_segments, columns=['StartTime','EndTime','Duration_sec'])

#過濾短暫停泊 (<30分鐘)
stops_df = stops_df[stops_df['Duration_sec'] >= 1800].reset_index(drop=True)

print("停泊區段：")
print(stops_df)

# 計算總停泊時間與航行時間
total_stop_time = stops_df['Duration_sec'].sum()
total_time = (df_filtered['ReceiveTime'].max() - df_filtered['ReceiveTime'].min()).total_seconds()
sailing_time = total_time - total_stop_time

print(f"總停泊時間 (hr): {total_stop_time/3600:.2f}")
print(f"航行時間 (hr): {sailing_time/3600:.2f}")
print(f"停泊比例: {total_stop_time/total_time:.2%}")
print(f"航行比例: {sailing_time/total_time:.2%}")

# 畫停泊/航行時間比例圖
plt.figure(figsize=(6,6))
plt.pie([total_stop_time, sailing_time], labels=['停泊', '航行'], autopct='%1.1f%%', colors=['salmon','skyblue'], startangle=90)
plt.title('停泊/航行時間比例')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 確保 ReceiveTime 是 datetime
df['ReceiveTime'] = pd.to_datetime(df['ReceiveTime'], errors='coerce')
df['ReceiveTime'] = pd.to_datetime(df['ReceiveTime'], errors='coerce')
df1_filtered = df1[df1['MMSI'] == 416426000]

# 篩選 2024/05/13 當天的資料
start_time = pd.Timestamp("2024-05-13 00:00:00")
end_time   = pd.Timestamp("2024-05-13 23:59:59")
df_day = df[(df['ReceiveTime'] >= start_time) & (df['ReceiveTime'] <= end_time)]

# 畫 Sog 時間序列圖
plt.figure(figsize=(15,4))
plt.plot(df_day['ReceiveTime'], df_day['Sog'], marker='o', linestyle='-', markersize=2, color='salmon')
plt.xlabel('ReceiveTime')
plt.ylabel('Sog (knots)')
plt.title('Sog 時間序列 (2024-05-13)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# 確保 ReceiveTime 是 datetime
df1 = pd.read_csv(r"C:\Users\slab\Desktop\Stage1\data\Device_AB00006.csv", low_memory = False)
df1['ReceiveTime'] = pd.to_datetime(df1['ReceiveTime'], errors='coerce')
df1_filtered = df1[df1['MMSI'] == 416464000]

# 篩選 2024/05/13 當天的資料


# 畫 Sog 時間序列圖
plt.figure(figsize=(15,4))
plt.plot(df1_filtered['ReceiveTime'], df1_filtered['Sog'], marker='o', linestyle='-', markersize=2, color='salmon')
plt.xlabel('ReceiveTime')
plt.ylabel('Sog (knots)')
plt.title('Sog 時間序列 (2024-05-13)')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()